### selenium installation

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 7.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

### data scraping

In [2]:
saglik_url = 'https://covid19.saglik.gov.tr/TR-66935/genel-koronavirus-tablosu.html'

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(saglik_url)

In [3]:
elem_cols = wd.find_element_by_tag_name('thead')
elem_rows = wd.find_element_by_tag_name('tbody')

In [4]:
cols = [col.text for col in elem_cols.find_elements_by_tag_name('th')]
rows = [[data.text for data in row.find_elements_by_tag_name('td')] for row in elem_rows.find_elements_by_tag_name('tr')]

### data transformation

In [5]:
import pandas as pd

df_covid = pd.DataFrame(data=rows, columns=cols)
df_covid.head()

,Tarih,Toplam Test Sayısı,Toplam Vaka Sayısı,Toplam Vefat Sayısı,Hastalarda Zatürre Oranı (%),Ağır Hasta Sayısı,Toplam İyileşen Hasta Sayısı,Bugünkü Vaka Sayısı,Bugünkü Hasta Sayısı,Bugünkü Test Sayısı,Bugünkü Vefat Sayısı,Bugünkü İyileşen Sayısı
0,29 HAZİRAN 2021,60.558.812,5.420.156,49.687,"3,7",730,5.287.094,5.846,510,229.846,53,6.536
1,28 HAZİRAN 2021,60.328.966,5.414.310,49.634,"2,9",751,5.280.558,5.283,431,223.525,58,5.327
2,27 HAZİRAN 2021,60.105.441,5.409.027,49.576,"2,9",736,5.275.231,4.883,390,215.294,52,5.937
3,26 HAZİRAN 2021,59.890.147,5.404.144,49.524,"2,9",733,5.269.294,5.266,470,222.561,51,7.402
4,25 HAZİRAN 2021,59.667.586,5.398.878,49.473,"2,9",737,5.261.892,5.630,477,222.936,56,7.184


In [6]:
df_covid = df_covid.replace(regex={'\.': ''})
df_covid = df_covid.replace(regex={',': '.'})

months = {
    'OCAK': '01', 'ŞUBAT': '02', 'MART': '03', 'NİSAN': '04', 
    'MAYIS': '05', 'HAZİRAN': '06', 'TEMMUZ': '07', 'AĞUSTOS': '08',
    'EYLÜL': '09', 'EKİM': '10', 'KASIM': '11', 'ARALIK': '12'
}

df_covid['Tarih'] = df_covid['Tarih'].replace(regex=months)
df_covid['Tarih'] = df_covid['Tarih'].replace(regex={'(\d) (\d)': r'\1-\2'})
df_covid['Tarih'] = df_covid['Tarih'].str.strip()
df_covid['Tarih'] = pd.to_datetime(df_covid['Tarih'], format='%d-%m-%Y')

df_covid = df_covid.set_index('Tarih')

df_covid.head()

,Toplam Test Sayısı,Toplam Vaka Sayısı,Toplam Vefat Sayısı,Hastalarda Zatürre Oranı (%),Ağır Hasta Sayısı,Toplam İyileşen Hasta Sayısı,Bugünkü Vaka Sayısı,Bugünkü Hasta Sayısı,Bugünkü Test Sayısı,Bugünkü Vefat Sayısı,Bugünkü İyileşen Sayısı
Tarih,,,,,,,,,,,
2021-06-29,60558812,5420156,49687,3.7,730,5287094,5846,510,229846,53,6536
2021-06-28,60328966,5414310,49634,2.9,751,5280558,5283,431,223525,58,5327
2021-06-27,60105441,5409027,49576,2.9,736,5275231,4883,390,215294,52,5937
2021-06-26,59890147,5404144,49524,2.9,733,5269294,5266,470,222561,51,7402
2021-06-25,59667586,5398878,49473,2.9,737,5261892,5630,477,222936,56,7184


### download

In [7]:
from google.colab import files

df_covid.to_csv('turkey.csv')
files.download('turkey.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>